In [2]:
import time
import csv
from pprint import pprint
import json
from batid.services.closest_bdg import get_closest
from batid.services.pairing_bdgs import pair

import os
from batid.models import Building
# necessary for SQL query execution from Jupyter
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

## Liste des colonnes

In [2]:
with open('data/data-es.csv') as f:
    reader = csv.DictReader(f, delimiter=';')
    
    pprint(reader.fieldnames)

["\ufeffNuméro de l'installation sportive",
 "Nom de l'installation sportive",
 'Numéro, type et nom de la voie',
 'Code postal',
 "Statut de la fiche d'enquête",
 "Etat de la fiche d'enquête",
 "Date de création de la fiche d'enquête",
 "Date de changement d'état de la fiche d'enquête",
 "Date de validation de la fiche d'enquête",
 "Accessibilité de l'installation en faveur des personnes en situation de "
 'handicap',
 "Date de l'enquête",
 "Accessibilité de l'installation en transport en commun",
 'Installation particulière',
 "Type de particularité de l'installation",
 "Accessibilité de l'installation en fonction du type handicap",
 "Accessibilité de l'installation en transport en commun des différents mode",
 'SIRET Installation',
 'Unité Administrative Immatriculée (UAI) Installation',
 "Numéro de l'équipement sportif",
 "Nom de l'équipement sportif",
 "Code du type d'équipement sportif",
 "Type d'équipement sportif",
 "Famille d'équipement sportif",
 "Etat de la fiche d'enquête d

## Vérifier l'unicité d'un identifiant

In [3]:
col_to_check = "Numéro de l'équipement sportif"

with open('data/data-es.csv') as f:
    reader = csv.DictReader(f, delimiter=';')
    
    identifiers = [row[col_to_check] for row in reader]
    unique_identifiers = set(identifiers)
    
    print(f"-- Vérification unicité du champ : {col_to_check}")
    
    print(f"Nb identifiants : {len(identifiers)}")
    print(f"Nb identifiants uniques : {len(unique_identifiers)}")
    
    if len(identifiers) != len(unique_identifiers):
        print('ALERTE : les identifiants ne sont pas uniques')
    else:
        print('OK : tous les identifiants sont uniques')

-- Vérification unicité du champ : Numéro de l'équipement sportif
Nb identifiants : 86082
Nb identifiants uniques : 86082
OK : tous les identifiants sont uniques


On trouve "Numéro de l'équipement sportif" comme identifiant unique

## Fonctions utilitaires

In [3]:
from batid.utils.misc import is_float

# sidenote : je me rends compte que je suis repassé au camelCase sans m'en rendre compte :)

def rowToParams(row): 
    return {
        'ext_id': row["Numéro de l'équipement sportif"],
        'lat': float(row['Latitude (WGS84)']) if is_float(row['Latitude (WGS84)']) else None,
        'lng': float(row['Longitude (WGS84)']) if is_float(row['Longitude (WGS84)']) else None,
        'radius': 20
    }

def estimatedTotalTime(avgPerLine):
    
    lines = 85000
    seconds = avgPerLine * lines
    
    return round(seconds / 3600, 2)

def extractSample(size):
    
    with open('data/data-es.csv') as f:
        reader = csv.DictReader(f, delimiter=';')
        return list(reader)[:size]
    
def report(totalTime, size):
    
    average = totalTime / size;
    
    print(f"Temps total : {totalTime:02f} secondes")
    print(f"Vitesse moyenne {round(average, 4)}  ms / ligne")
    print(f"Estimation temps total : {estimatedTotalTime(average)} heures")
    print("nb : le temps total d'execution du script peut être plus long qu'indiqué en raison de la lecture du CSV (qq secondes) qui n'est pas prise en compte")


## Test de vitesse en séquentiel

In [36]:
rows_to_check = 50
data = extractSample(rows_to_check)

start = time.perf_counter()
for row in data:
    params = rowToParams(row)
    if params['lat'] and params['lng']:
        closest = get_closest(params['lat'], params['lng'], params['radius'])

end = time.perf_counter()
elapsed = end - start

report(elapsed, rows_to_check)


Temps total : 9.521135 secondes
Vitesse moyenne 0.1904  ms / ligne
Estimation temps total : 4.5 heures


## Test de vitesse avec asyncIO

In [49]:
import asyncio

async def async_get_closest_time(lat, lng, radius):
    return get_closest(lat,lng,radius)
    
rows_to_check = 50
data = extractSample(rows_to_check)

tasks = []
start = time.perf_counter()
for row in data:
    params = rowToParams(row)
    if params['lat'] and params['lng']:
        tasks.append(asyncio.create_task(async_get_closest_time(params['lat'], params['lng'], params['radius'])))
    
await asyncio.gather(*tasks)
end = time.perf_counter()
elapsed = end - start

report(elapsed, rows_to_check)

Temps total : 17.164769 secondes
Vitesse moyenne 0.3433  ms / ligne
Estimation temps total : 8.11 heures


Le temps mis en asyncIO est bien moins bon qu'en séquentiel. On peut tester en multiThreads

In [ ]:
## Test de vitesse en multithreads

In [57]:
import concurrent.futures

rows_to_check = 2000
data = extractSample(rows_to_check)

tasks = []
results = []

start = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    
    for row in data:
        params = rowToParams(row)
        if params['lat'] and params['lng']:
            tasks.append(executor.submit(get_closest, **params))
    

    for future in concurrent.futures.as_completed(tasks):
        bdg = future.result()
        if bdg:
            results.append((bdg.rnb_id, bdg.distance))
        continue

end = time.perf_counter()
elapsed = end - start

report(elapsed, rows_to_check)

print('--')
print(f"Found {len(rnb_ids)} matches")
print(results[10])

Temps total : 9.123574 secondes
Vitesse moyenne 0.0046  ms / ligne
Estimation temps total : 0.11 heures
--
Found 1796 matches
('Z5K1J9PFWMGH', Distance(m=2.75066279))


L'approche multithread semble être la bonne. Le gain de perf est un facteur 70. L'estimation est une moyenne.

# Rapprochement complet

In [14]:
rows_to_check = 1000
raw_data = extractSample(rows_to_check)


data = []
for row in raw_data: 
    params = rowToParams(row)
    if params['lat'] and params['lng']:
        data.append(params)

        
        
start = time.perf_counter()
result = pair(data, 'closest')
end = time.perf_counter()
elapsed = end - start

report(elapsed, rows_to_check)

Temps total : 5.782526 secondes
Vitesse moyenne 0.0058  ms / ligne
Estimation temps total : 0.14 heures
nb : le temps total d'execution du script peut être plus long qu'indiqué en raison de la lecture du CSV (qq secondes) qui n'est pas prise en compte
